# Regression (Chapter 7)

This module focuses on a particular class of supervised machine learning: regression, where we are trying to discover relationships between variables in order to predict continuous quantities. Neelima gave you an introduction to linear regression, logistic regression, and how to use scikit-learn to accomplish both tasks. Content for this module:
* linear least-squares 
* non-linear regression models
* penalized regression models
* constrained regression

Resources:

1. Gander, Gander and Kwok (2014), Scientific Computing: An introduction using Maple and MATLAB, Springer, https://www.springer.com/us/book/9783319043241
2. Kuhn and Johnson (2013), Applied Predictive Modeling, Springer
https://www.springer.com/us/book/9781461468486

# Linear regression

For samples with $D$ features, linear regression attempts to fit a $D$-dimensional hyperplane to the data.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

n = 200
x = np.random.rand(n,1) # feature 1
y = np.random.rand(n,1) # feature 2
z = x + 0.5*y + 0.1*np.random.randn(n,1) + 0.2 # response variable, z(x,y)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x,y,z,c='b',marker='o')
ax.set_xlabel('feature 1')
ax.set_ylabel('feature 2')
_ = ax.set_zlabel('response')

Specifically, we are trying to find coefficients $a_0 + a_1, a_2, \ldots, a_D$ so that the hyperplane

\begin{align}
 z(x_1,x_2,\ldots,x_D) = a_0 + a_1 x_1 + a_2 x_2 + \cdots + a_D x_D
\end{align}

models the response variable. (note, this is the generalize form of equation (6.2) in textbook).  Suppose we have $N$ samples, each with $D$ features, $\vec{x}_i = [{x_i}_1,{x_i}_2,\ldots,{x_i}_D],\, i= 1,\ldots N$. Then the hope, is that 

\begin{align}
  z({x_i}_1,{x_i}_2,\ldots,{x_i}_D) \approx y_i,\quad \text{for } i = 1,\ldots, N
\end{align}

This gives rise to an over-determined system of equations:

\begin{align}
  a_0 + a_1 x_{11} + a_2 x_{12} + \cdots + a_D x_{1D} &\approx y_1\\
  a_0 + a_1 x_{21} + a_2 x_{22} + \cdots + a_D x_{2D} &\approx y_2\\
    &\vdots \\
  a_0 + a_1 x_{N1} + a_2 x_{N2} + \cdots + a_D x_{ND} &\approx y_N\\
\end{align}

This is more easily expressed in matrix form, $X\,a = y$,

\begin{align}
\begin{bmatrix}
1 & x_{11} & x_{12} & \cdots & x_{1D}\\
1 & x_{21} & x_{22} & \cdots & x_{2D}\\
1 & x_{31} & x_{32} & \cdots & x_{3D}\\
\vdots & \vdots & \vdots & \vdots & \vdots \\
1 & x_{N1} & X_{x2} & \cdots & x_{ND}
\end{bmatrix}
\begin{bmatrix}
a_0\\
a_1 \\
\vdots\\
a_D
\end{bmatrix}
\approx
\begin{bmatrix}
y_1 \\
y_2 \\
y_3 \\
\vdots \\
y_N
\end{bmatrix}
\end{align}

Here, the matrix $X$ is of size: $N \times (D+1)$, the vector of coefficients we seek is of size $(D+1)\times 1$, the right-hand side vector is of size $N \times 1$.  Here, we are imagining that $N \gg D$.  How do we solve the overdetermined system?

Well, it turns out we can't exactly most of the time, i.e., we can not find $a$ such that $X\,a =y$.  There is always some mismatch, a residual: $r = X\,a - y$.  We want to minimize this residual, for example, minimize $\|r\|^2_2 = \|X\,a - y\|^2_2$, i.e.,

$$ \min \|X\,a - y \|^2_2 = \min \left( \sum_{i = 1}^N  (X\,a - y)_i^2 \right) $$

### Theorem

If $X^T (X\,a - y) = 0$, then $a$ solves the linear least squares problem, i.e. $a$ minimizes $\|X\,a - y\|_2^2$

Proof: Let $c$ be any vector of size $(D+1)\times 1$.  Then $X(a+c) - y = X\,a -y + X\,c $, and

\begin{align}
\|X(a+c) - y\|_2^2 &= \left( X\,a -y + X\,c \right)^T \left(X\,a -y + X\,c \right) \\
&= (X\,a - y)^T (X\,a -y) + 2 (X\,c)^T(X\,a - y) + (X\,c)^T (X\,c) \\
&= \|X\,a - y\|_2^2 + 2c^T X^T (X\,a - y) + \|X\,c\|_2^2
\end{align}

If $X^T (X\,a - y) = 0$, then
\begin{align}
\|X(a+c) - y\|_2^2 = \|X\,a - y\|_2^2 + \|X\,c\|_2^2
\end{align}

Hence, for every $c$, we have 
\begin{align}
\|X(a+c) - y\|_2^2 \ge  \|X\,a - y\|_2^2
\end{align}

so $a$ must minimize $\|X\,a - y\|_2^2$ as required. 

The equation $X^T(X\,a - y) = 0$ is often written as

$$ X^T X\,a = X^T y, $$

called the normal equations.  There is a geometric interpretation: the vector in the range (column space of $X$) that lies closest to $y$ makes $X\,a - y$ perpendicular to the range.  Note, 
* the normal equations expresses the $N\times(D+1)$ linear least squares problem as a $(D+1)\times (D+1)$ linear system.
* the matrix $X^T X$ is symmetric
* $X^T X$ is singular . if and only if the columns of $X$ are linearly dependent, i.e., the rank of $X$ is less than $(D+1)$
* if $X^T X$ is non singular, then it is positive definite.

Observe that if $X^T X$ is non singular, then
$$ X^T X\,a = X^T y, $$
can be written as 
$$ a = (X^T X)^{-1} X^T y, $$

One sometimes defines the pseudo inverse,
$$ X^\dagger = (X^T X)^{-1} X^T.$$
Then, the least squares problem $X\,a = y$ has a solution $a = X^\dagger y$.


Numerically, one never forms the pseudo inverse to solve the least squares problem.  Rather, we rely on a useful factorization known as the QR factorization.  Every matrix has a QR factorization, $X = QR$, where $Q$ is an orthogonal matrix, and $R$ is upper triangular.  For convenience, lets refer to
* $X$ as an $m \times n$ matrix
* $Q$ is an $m \times m$ matrix, whose columns are orthonormal to each other, i.e.:
    * $q_i \cdot q_j = 0$ if $i\neq j$
    * $q_i \cdot q_i = 1, \quad i=1,2,\ldots,m$
* $R$ is an $m \times n$ upper triangular matrix, i.e., $r_{ij} = 0$ if $i > j$.

Lets switch to a toy example where we have an over-determined system, and explore the QR factorization and least-squares solution.  Suppose we have two packages that we wish to ship.  We measure the weight of these two packages in the office.  Package $A$ measures in at 2 pounds, package $B$ measures in at 5 pounds.  At the distribution site however, the two packages are weighed together, and the joint weight is reported at 8 pounds.  Use a least-squares solution to find a best estimate for the weight of each package.

In [11]:
import numpy as np
X = np.array( [ [1,0], [0,1], [1,1]])
print("our matrix X:")
print(X)

our matrix X:
[[1 0]
 [0 1]
 [1 1]]


In [12]:
[Q,R] = np.linalg.qr(X, mode = 'complete')
print("Lets check that Q*R = X, up to machine precision")
print(np.matmul(Q,R))

Lets check that Q*R = X, up to machine precision
[[ 1.00000000e+00 -2.22044605e-16]
 [ 0.00000000e+00  1.00000000e+00]
 [ 1.00000000e+00  1.00000000e+00]]


Lets check that $Q$ has orthonormal columns

In [7]:
print("matrix Q: ")
print(Q)
print("\nVarious inner products:")
print("q1^t q_1 = %g\n " % np.dot(Q[:,0],Q[:,0]) )
print("q1^t q_2 = %g \n" % np.dot(Q[:,0],Q[:,1]) )
print("q1^t q_3 = %g \n" % np.dot(Q[:,0],Q[:,2]) )
print("q2^t q_2 = %g \n" % np.dot(Q[:,1],Q[:,1]) )
print("q2^t q_3 = %g \n" % np.dot(Q[:,1],Q[:,2]) )
print("q3^t q_3 = %g " % np.dot(Q[:,2],Q[:,2]) )


matrix Q: 
[[-0.70710678  0.40824829 -0.57735027]
 [-0.         -0.81649658 -0.57735027]
 [-0.70710678 -0.40824829  0.57735027]]

Various inner products:
q1^t q_1 = 1
 
q1^t q_2 = 8.6508e-17 

q1^t q_3 = -1.05825e-16 

q2^t q_2 = 1 

q2^t q_3 = 0 

q3^t q_3 = 1 


Lets check that $R$ is upper triangular

In [8]:
print("R = ")
print(R)

R = 
[[-1.41421356 -0.70710678]
 [ 0.         -1.22474487]
 [ 0.          0.        ]]


There is a reduced factorization that is more useful. $ X = \hat{Q} \hat{R}$, where $\hat{Q}$ is an $m\times n$ matrix with orthonormal columns, and $\hat{R}$ is an $n\times n$ upper triangular matrix.

In [13]:
[Q,R] = np.linalg.qr(A, mode = 'reduced')
print("Q = ")
print(Q)
print("R = ")
print(R)
print("Lets check that Q*R = X, up to machine precision")
print(np.matmul(Q,R))

Q = 
[[-0.70710678  0.40824829]
 [-0.         -0.81649658]
 [-0.70710678 -0.40824829]]
R = 
[[-1.41421356 -0.70710678]
 [ 0.         -1.22474487]]
Lets check that Q*R = X, up to machine precision
[[ 1.00000000e+00 -2.22044605e-16]
 [ 0.00000000e+00  1.00000000e+00]
 [ 1.00000000e+00  1.00000000e+00]]


How do we use the $QR$ factorization to solve the normal equations?  Suppose we have $X = \hat{Q}\,\hat{R}$.  Observe:
\begin{align}
X^T X a &= X^T y \\
(\hat{Q}\,\hat{R})^T (\hat{Q}\,\hat{R})\, a &= (\hat{Q}\,\hat{R})^T y \\
\hat{R}^T \hat{Q}^T \hat{Q}\, \hat{R} \, a &= \hat{R}^T \hat{Q}^T y
\end{align}


If $\hat{R}^T$ exists, and observing that $\hat{Q}^T \hat{Q}$ gives an $n\times n$ identity matrix,
\begin{align}
&\implies \hat{R}^T \hat{R} \, a = \hat{R}^T \hat{Q}^T y \\
&\implies  \hat{R} \, a = \hat{Q}^T y
\end{align}

Lets use our toy example to fit a regression line, first using sci-kit learn, and then using our derived normal equations, simplified using the QR factorization.